In [1]:
import ast
import json
import nltk
import pandas as pd
from copy import deepcopy

from src.web_scraping import Digger
from src.recipe import Recipe

#nltk.download('punkt')
countries = {'greek':{'link':'https://www.recipesource.com/ethnic/europe/greek/indexall.html', 'recipes': []},
             'british': {'link': 'https://www.recipesource.com/ethnic/europe/british/indexall.html', 'recipes': []},
             'german': {'link': 'https://www.recipesource.com/ethnic/europe/french/indexall.html', 'recipes': []},
             'filipino':{'link': 'https://www.recipesource.com/ethnic/asia/filipino/indexall.html', 'recipes':[]},
             'indian':{'link': 'https://www.recipesource.com/ethnic/asia/indian/indexall.html', 'recipes': []},
             'indonesian': {'link': 'https://www.recipesource.com/ethnic/asia/indonesian/indexall.html', 'recipes': []},
             'cajun': {'link': 'https://www.recipesource.com/ethnic/americas/cajun/indexall.html', 'recipes': []},
             'carribean': {'link': 'https://www.recipesource.com/ethnic/americas/caribbean/indexall.html', 'recipes': []},
             'italian':{'link': 'https://www.recipesource.com/ethnic/europe/italian/indexall.html', 'recipes': []},
             'mexican': {'link': 'https://www.recipesource.com/ethnic/americas/mexican/indexall.html', 'recipes': []},
             'chinese':{'link': 'https://www.recipesource.com/ethnic/asia/chinese/indexall.html', 'recipes': []},
             'thai': {'link': 'https://www.recipesource.com/ethnic/asia/thai/indexall.html', 'recipes': []},
             'vietnamese': {'link': 'https://www.recipesource.com/ethnic/asia/vietnamese/indexall.html', 'recipes': []},
             'french':{'link': 'https://www.recipesource.com/ethnic/europe/french/indexall.html', 'recipes':[]},
             'japanese': {'link': 'https://www.recipesource.com/ethnic/asia/japanese/indexall.html', 'recipes': []},
             'irish':{'link':'https://www.recipesource.com/ethnic/europe/irish/indexall.html', 'recipes': []},
             'korean': {'link': 'https://www.recipesource.com/ethnic/asia/korean/indexall.html', 'recipes': []},
             'russian':{'link': 'https://www.recipesource.com/ethnic/europe/russian/indexall.html', 'recipes': []}}
    
#data_leak_terms = ['greek', 'german', 'filipino', 'indian', 'indonesian', 'cajun', 'carribean', 'italian', 'mexican',
#                  'chinese', 'thai', 'vietnamese', 'french', 'japanese', 'irish', 'koren', 'russian', '\r', "b'", 'b"' '*',
#                  '(tm)', 'categories', 'meal master']

In [40]:
#with open('web_scrape.json', 'r') as f:
#    countries = json.load(f)

#dig = Digger(countries)
for c in dig.countries:
    print(c, ": ", len(dig.countries[c]['recipes']))

greek :  407
british :  92
german :  110
filipino :  54
indian :  589
indonesian :  112
cajun :  540
carribean :  87
italian :  657
mexican :  768
chinese :  0
thai :  0
vietnamese :  0
french :  0
japanese :  0
irish :  0
korean :  0
russian :  0


In [38]:
dig.dig_soup('mexican')


mexican
mexican :  768


In [39]:
for i in range(10):
    print('\n------------------------------------------')
    print(dig.countries['mexican']['recipes'][i]) 


------------------------------------------
acapulco chicken (en escabeche) poultry yield 6 servings 2 c unsalted chicken broth 1/2 yellow bell pepper sliced defatted 2 tb minced jalapeno chili with 1 tb olive oil seeds 2 ts ground cumin 1 onion halved thinly sliced 2 tb pickling spice 1/3 c rice wine vinegar 1/2 red bell pepper sliced 1/4 c fresh cilantro leaves 1 lb boneless chicken breast 3 lg garlic cloves minced halves baked (no oil) tortilla chips boil broth pickling spice heavy large saucepan ten minutes. strain return liquid saucepan. add chicken onion vinegar garlic oil cumin pan. simmer over very low heat until chicken just cooked through ten minutes. transfer chicken onions shallow dish. top with bell peppers minced chilli. boil cooking liquid until reduced 2/3 c ten minutes. pour liquid over chicken let cool 30 minutes. add cilantro chicken mixture. cover refrigerate until well chilled turning chicken occasionally 4 hours (can be prepared one day ahead). slice chicken trans

NameError: name 'dig' is not defined

In [41]:
#print(len(dig.countries['chinese']['recipes']))
#with open('web_scrape.json', 'w') as outfile:
#    json.dump(dig.countries, outfile)

In [ ]:
def build_training_set(filename, countries):
    
    with open(filename, 'r') as f:
        train_r = json.load(f)

    for key, val in train_r.items():
        for c in countries:
            if 'title' in train_r[key]:
                try:
                    train_r[key]['title'].lower()
                    instructions = train_r[key]['instructions'].lower().replace('\n', '')
                    ingredients = ' '.join(train_r[key]['ingredients'])
                except AttributeError:
                    continue
                if c in train_r[key]['title'].lower():
                    countries[c].append(' '.join([ingredients, instructions]).replace('advertisement', ''))
            

In [ ]:
filenames = ['data/recipes_raw_nosource_ar.json', 'data/recipes_raw_nosource_epi.json', 'data/recipes_raw_nosource_fn.json']

for f in filenames:
    build_training_set(f, countries)


In [ ]:
sum = 0
for c in countries:
    sum += len(countries[c])
    print(c, ': ', len(countries[c]))
print(sum)

In [ ]:
fish = ['anchovies', 'barracuda', 'basa', 'bass', 'cod', 'sablefish', 'blowfish', 'bluefish',
        'bombay duck', 'bream', 'brill', 'butter fish', 'catfish', 'dogfish', 'dorade', 'eel',
        'flounder', 'grouper', 'haddock', 'hake', 'halibut', 'herring', 'ilish', 'john dory', 'lamprey',
        'lingcod', 'mackerel', 'mahi mahi', 'monkfish', 'mullet', 'orange roughy', 'parrotfish', 'patagonian toothfish',
        'perch', 'pike', 'pilchard', 'pollock', 'pomfret', 'pompano', 'sablefish', 'salmon', 'sanddab', 'sardine',
        'sea bass', 'shad', 'alewife', 'shark', 'skate', 'smelt', 'snakehead', 'snapper', 'rockfish', 'sole', 'sprat', 
        'sturgeon', 'surimi', 'swordfish', 'tilapia', 'tilefish', 'trout', 'tuna', 'albacore', 'yellowfin', 'bigeye',
        'bluefin', 'dogtooth', 'turbot', 'wahoo', 'whitefish', 'whiting', 'witch', 'righteye', 'whitebait', 'roe', 'caviar',
        'ikura', 'kazunoko', 'lumpfish','masago', 'capelin', 'tobiko', 'flying-fish', 'shellfish', 'crustaceans', 'crab',
        'dungeness', 'crayfish', 'langostino', 'lobster', 'shrimp', 'prawns', 'mollusca', 'cockle', 'cuttlefish', 'clam',
        'loco', 'mussel', 'octopus', 'oyster', 'periwinkle', 'scallop', 'squid', 'escargot', 'snails', 'nautilus', 'dolphin',
        'seal', 'whale', 'echinoderms', 'sea cucumber', 'uni', 'sea urchin', 'roe', 'medusozoa', 'Tunicates', 'sea squirts',
        'microcosmus', 'sabatieri', 'sea fig', 'sea violet', 'crabmeat']

meat = ['chicken', 'turkey', 'duck', 'mallard', 'goose','greylag', 'guinea', 'fowl', 'pigeon', 'dove', 'quail',
        'partridge', 'swiftlet', 'swan', 'beef', 'steak', 'sirloin', 'veal', 'brisket', 'pork', 'ham', 'bacon', 'shank', 'lamb',
        'mutton', 'goat', 'chuck']

keyword = ['meat', 'seafood', 'vegetarian', 'vegan']


In [ ]:
import json
with open('data/train.json/train.json', 'r') as f:
    nationalities = json.load(f)
print(type(nationalities))
limited = nationalities#[:10000]

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
train, test = train_test_split(limited, test_size=0.2, random_state=4622)


In [ ]:
train_dict = {'ingredients':[], 
              'cuisine': []}
for x in train:
    train_dict['ingredients'].append(' '.join(x['ingredients']))
    train_dict['cuisine'].append(x['cuisine'])
    
test_dict = {'ingredients':[], 
              'cuisine': []}
for x in test:
    test_dict['ingredients'].append(' '.join(x['ingredients']))
    test_dict['cuisine'].append(x['cuisine'])

In [ ]:
X_train = train_dict['ingredients']
X_test = test_dict['ingredients']
y_train = train_dict['cuisine']
y_test = test_dict['cuisine']

len(X_train),len(y_train),len(X_test),len(y_test)


skf = StratifiedKFold(n_splits=3, shuffle=False, random_state=None)
splits = skf.get_n_splits(X_train, y_train)

In [ ]:
import numpy as np
from sklearn import linear_model, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn import linear_model, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score

In [ ]:
# Create logistic regression
logistic = linear_model.LogisticRegression()

# Create regularization penalty space
penalty = ['l2']

# Create regularization hyperparameter space
C = np.logspace(0, 1.5, 5)

solver = ['liblinear', 'newton-cg', 'lbfgs']


class_weight = ['balanced']

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty, solver=solver, class_weight=class_weight)


In [ ]:
pipeline = Pipeline([('clf',linear_model.LogisticRegression())])


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf.shape)


le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train_transform = le.transform(y_train)
print(len(y_train_transform))

In [ ]:
# Create grid search using 5-fold cross validation
clf = GridSearchCV(logistic, hyperparameters, cv=splits, verbose=3)

# Fit grid search
best_model = clf.fit(X_train_counts, y_train_transform)

# View best hyperparameters
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

In [ ]:
print(best_model.best_params_)
print(best_model.best_score_)


In [ ]:
from joblib import dump, load

dump(rf_random, 'random_tree.joblib') 
clf = load('random_tree.joblib') 
